### Run this in Python kernel

In [ ]:
%%html
<style>
body {
    font-family: "Arial", cursive, sans-serif; # font style of application
}
pre {
    font-family: "Courier New"; # font style of cell outputs
}
</style>  

### Run below in the backtesting_clojure kernel

In [1]:
; import libraries from kernel
(ns clojure-backtesting.demo
  (:require [clojure.test :refer :all]
            [clojure-backtesting.data :refer :all]
            [clojure-backtesting.data-management :refer :all]
            [clojure-backtesting.order :refer :all]
            [clojure-backtesting.evaluate :refer :all]
            [clojure-backtesting.plot :refer :all]
            [clojure-backtesting.counter :refer :all]
            [clojure-backtesting.portfolio :refer :all]
            [clojure-backtesting.parameters :refer :all]
            [clojure-backtesting.large-data :refer :all]
            [clojure.string :as str]
            [clojure.pprint :as pprint]
            [java-time :as t]
            [clojupyter.kernel.version :as ver]
            [clojupyter.misc.helper :as helper]
  ) ;; require all libriaries from core
  (:use clojure.pprint)
)

nil

### Import dataset

In [2]:
; path to dataset = "../resources/CRSP-extract.csv"
; change it to the relative to your own dataset
;
(reset! data-set (add-aprc (read-csv-row "../resources/CRSP-extract.csv")));

### Initialise portfolio （Go back here everytime you want to restart.）

In [3]:
;; initialise with current date and initial capital (= $10000)
(init-portfolio "1980-12-15" 1000);

### Write a strategy

The following code implements a trading strategy called Golden Rule:

MA 50 cross above the MA 200 (golden cross)

MA 50 cross below the MA 200 (death cross)

So in the codes, MA50 and MA200 are compared on a daily basis, if golden cross occurs, then you set a buy order; if death cross occurs, then you set a sell order first 



In [ ]:
(def MA50-vec-aapl [])
(def MA200-vec-aapl [])
(def MA50-vec-f [])
(def MA200-vec-f [])
(while (not= (get-date) "2000-12-26")
    (do
      (def tics (deref available-tics)) ;20 ms
      (def MA50-vec-aapl (get-prev-n-days :PRC 50 "AAPL" MA50-vec-aapl))
      (def MA200-vec-aapl (get-prev-n-days :PRC 200 "AAPL" MA200-vec-aapl))
      (def MA50-vec-f (get-prev-n-days :PRC 50 "F" MA50-vec-f ))
      (def MA200-vec-f (get-prev-n-days :PRC 200 "F" MA200-vec-f))
      (let [[MA50 MA200] [(moving-average :PRC MA50-vec-aapl) (moving-average :PRC MA200-vec-aapl)]]
        (if (> MA50 MA200)
          (order "AAPL" 0.1 :print false) 
          (order "AAPL" 0 :remaining true))))
      (let [[MA50 MA200] [(moving-average :PRC MA50-vec-f) (moving-average :PRC MA200-vec-f)]]
        (if (> MA50 MA200)
          (order "F" 1  :print false) 
          (order "F" 0 :remaining true )))
      (update-eval-report (get-date))
      (next-date))
(end-order)

In [5]:
;(pprint/print-table (take 200 (deref portfolio-value)))
(take 10 (deref order-record))

({:date "1981-02-27", :tic "AAPL", :price 26.625, :aprc "26.39", :quantity 0.1} {:date "1981-02-27", :tic "F", :price 19.625, :aprc "103.35", :quantity 1} {:date "1981-03-02", :tic "AAPL", :price 26.6875, :aprc "26.42", :quantity 0.1} {:date "1981-03-02", :tic "F", :price 20.125, :aprc "104.49", :quantity 1} {:date "1981-03-03", :tic "AAPL", :price 26.3125, :aprc "26.26", :quantity 0.1} {:date "1981-03-03", :tic "F", :price 20.25, :aprc "104.77", :quantity 1} {:date "1981-03-04", :tic "AAPL", :price 26.0625, :aprc "26.15", :quantity 0.1} {:date "1981-03-04", :tic "F", :price 20.25, :aprc "104.77", :quantity 1} {:date "1981-03-05", :tic "AAPL", :price 25.9375, :aprc "26.09", :quantity 0.1} {:date "1981-03-05", :tic "F", :price 20.125, :aprc "104.49", :quantity 1})

### Check portfolio record

In [6]:
;; view final portfolio
(view-portfolio)


| :asset | :price | :aprc | :quantity | :tot-val |
|--------+--------+-------+-----------+----------|
|   cash |    N/A |   N/A |       N/A |    46366 |


nil

In [7]:
;; view portfolio value and return
(view-portfolio-record 10)


|      :date | :tot-value | :daily-ret | :tot-ret | :loan | :leverage |
|------------+------------+------------+----------+-------+-----------|
| 1980-12-15 |      $1000 |      0.00% |    0.00% | $0.00 |     0.00% |
| 1981-02-27 |      $1000 |      0.00% |    0.00% | $0.00 |     0.00% |
| 1981-03-02 |      $1001 |      0.00% |    0.05% | $0.00 |     0.00% |
| 1981-03-03 |      $1001 |      0.00% |    0.07% | $0.00 |     0.00% |
| 1981-03-04 |      $1001 |      0.00% |    0.07% | $0.00 |     0.00% |
| 1981-03-05 |      $1000 |      0.00% |    0.02% | $0.00 |     0.00% |
| 1981-03-06 |      $1011 |      0.00% |    0.50% | $0.00 |     0.00% |
| 1981-03-09 |      $1028 |      0.00% |    1.23% | $0.00 |     0.00% |
| 1981-03-10 |      $1017 |      0.00% |    0.74% | $0.00 |     0.00% |
| 1981-03-11 |      $1019 |      0.00% |    0.82% | $0.00 |     0.00% |


nil

### Generate evaluation report

In [8]:
(eval-report 434)
;; (update-eval-report)
;; (count(deref eval-report-data))
;; (deref dailyret-record)


|      :date | :tot-val |                                                                                                                                                                                                                                                     :vol |     :sharpe | :pnl-pt |
|------------+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+---------|
| 1981-02-26 |    $1000 |                                                                                                                                                                                                                                                    0.00% |       0.00% |      $0 |
| 1981-02-27 |    $1001 |                                                                       

| 1981-04-03 |     $936 |                                                                                                                                                                                                                                                    0.00% |   -7464.43% |     $-1 |
| 1981-04-06 |     $958 |                                                                                                                                                                                                                                                    0.00% |   -5453.43% |     $-1 |
| 1981-04-07 |     $958 |                                                                                                                                                                                                                                                    0.00% | -193088.12% |     $-1 |
| 1981-04-08 |     $988 |                                                                        

| 1981-05-14 |    $1154 |                                                                                                                                                                                                                                                     NaN% |        NaN% |      $2 |
| 1981-05-15 |    $1125 |                                                                                                                                                                                                                                                     NaN% |        NaN% |      $1 |
| 1981-05-18 |    $1125 |                                                                                                                                                                                                                                                     NaN% |        NaN% |      $1 |
| 1981-05-19 |    $1081 |                                                                        

| 1981-06-24 |    $1201 |                                                                                                                                                                                                                                                     NaN% |        NaN% |      $1 |
| 1981-06-25 |    $1223 |                                                                                                                                                                                                                                                     NaN% |        NaN% |      $2 |
| 1981-06-26 |    $1179 |                                                                                                                                                                                                                                                     NaN% |        NaN% |      $1 |
| 1981-06-29 |    $1199 |                                                                        

| 1981-08-04 |     $712 |                                                                                                                                                                                                                                                     NaN% |        NaN% |     $-1 |
| 1981-08-05 |     $712 |                                                                                                                                                                                                                                                     NaN% |        NaN% |     $-1 |
| 1981-08-06 |     $681 |                                                                                                                                                                                                                                                     NaN% |        NaN% |     $-1 |
| 1981-08-07 |     $618 |                                                                        

...

| 1982-06-23 |     $626 |                                                                                                                                                                                                                                                     NaN% |        NaN% |     $-1 |
| 1982-06-24 |     $538 |                                                                                                                                                                                                                                                     NaN% |        NaN% |     $-1 |
| 1982-06-25 |     $568 |                                                                                                                                                                                                                                                     NaN% |        NaN% |     $-1 |
| 1982-06-28 |     $598 |                                                                        

| 1982-08-03 |     $382 |                                                                                                                                                                                                                                                     NaN% |        NaN% |     $-2 |
| 1982-08-04 |     $268 |                                                                                                                                                                                                                                                     NaN% |        NaN% |     $-2 |
| 1982-08-05 |     $268 |                                                                                                                                                                                                                                                     NaN% |        NaN% |     $-2 |
| 1982-08-06 |     $245 |                                                                        

| 1982-09-13 |    $1403 |                                                                                                                                                                                                                                                     NaN% |        NaN% |      $1 |
| 1982-09-14 |    $1376 |                                                                                                                                                                                                                                                     NaN% |        NaN% |      $1 |
| 1982-09-15 |    $1376 |                                                                                                                                                                                                                                                     NaN% |        NaN% |      $1 |
| 1982-09-16 |    $1238 |                                                                        

| 1982-10-21 |    $2293 |                                                                                                                                                                                                                                                     NaN% |        NaN% |      $3 |
| 1982-10-22 |    $1717 |                                                                                                                                                                                                                                                     NaN% |        NaN% |      $2 |
| 1982-10-25 |    $1815 |                                                                                                                                                                                                                                                     NaN% |        NaN% |      $2 |
| 1982-10-26 |    $1881 |                                                                        

nil

In [9]:
(take 1 (deref eval-report-data))

({:date "1981-02-26", :tot-val "$1000", :vol "0.00%", :sharpe "0.00%", :pnl-pt "$0"})

### Plot variables

In [ ]:
(def data (deref portfolio-value))

In [ ]:
; Add legend name to series
(def data-to-plot
 (map #(assoc % :plot "portfolio")
  data))

In [ ]:
(first data-to-plot)

In [ ]:
(plot data-to-plot :plot :date :daily-ret)